# 예측을 위한 EDA (DieTanic)


EDA(Exploratory data analysis, 탐색적 자료 분석): 수집한 데이터를 다양한 관점에서 관찰하고 이해하는 과정, 그래프나 통계적인 방법으로 자료를 직관적으로 바라보는 과정이다.

과정: 데이터 가져오기 - 데이터 모양 확인 - 데이터 타입 확인 - 데이터 기초 분석 - 데이터 클린징 - 데이터 시각화 - 의사결정 EDA

4가지 주제
1. 저항성: 자료의 일부가 완전히 다른 값으로 대체되거나 파손되었을 때, 영향받는 정도. 저항성이 높으면, 일부가 훼손되더라도, 유의미한 정보 검출 가능
2. 잔차의 해석: 잔차는 주경향으로 부터 얼마나 벗어났는 지를 말해준다. 비정상적인 수치가 나왔을 때, 그 원인을 파악하는 것
3. 자료의 재표현: 자료 분석을 단순화하기 위해, 로그, 제곱, 역수 등을 취해 적당한 척도로 바꾸는 것
4. 자료의 현시성: 자료의 정보를 시각적으로 표현하여, 구조를 효율적으로 판단하게 해주는 것


### *때때로 삶은 가능한 최악의 순간에 원하는 것을 주는 잔인한 유머 감각이 있다.*
                                                                                       -리사 클레이파스

                                                                                                                                     

타이타닉의 침몰은 역사에서 가장 악명높은 난파선 중 하나이다. 1912년 4월 15일, 첫 출항에서, 타이타닉은 빙산과 충돌 후에 가라앉았고, 2224명의 승객 중 1502명의 승객과 선원이 죽었다. 이것이 타이타닉을 **다이(Die)타닉**인 이유이다. 아무도 잊을 수 없고, 정말 잊지 못 할 재앙이다.

타이타닉을 짓는 데, 750만 달러가 소요되었는데, 충돌 때문에 바다에 가라앉았다. 타이타닉 Dataset은 초보자들이 데이터 사이언스에 첫 시작 및 kaggle에서 competitions에 참여하기 매우 좋은 Dataset이다.

이 notebook의 목적은 **예측가능한 모델링 문제에서 어떻게 작업 흐름을 가져갈 지에 대한 아이디어**를 주는 것이다. 어떻게 우리는 feature를 확인하고, 어떻게 우리는 새로운 feature와 어떤 머신 러닝 개념을 추가할 것인가. 나는 처음 하는 사람도 모든 구문을 이해할 수 있게 하기 위해 가능한 기초적인 notebook을 유지하려고 노력했다.

만약 notebook이 좋고, 도움이 된다면, **upvote를 부탁한다**. 이것은 나에게 동기부여가 된다.

## Notebook 내용:

#### Part1: Exploratory Data Analysis(EDA):
1)Analysis of the features.

2)Finding any relations or trends considering multiple features.
#### Part2: Feature Engineering and Data Cleaning:
1)Adding any few features.

2)Removing redundant features.

3)Converting features into suitable form for modeling.
#### Part3: Predictive Modeling
1)Running Basic Algorithms.

2)Cross Validation.

3)Ensembling.

4)Important Features Extraction.

## Part1: Exploratory Data Analysis(EDA)

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
data=pd.read_csv('../input/train.csv')

In [ ]:
data.head()

In [ ]:
data.isnull().sum() #null 값들의 개수를 확인

**Age, Cabin 과 Embarked** 이 null 값들을 갖고 있으므로, 고칠 것이다.

### 얼마나 살아남았나??

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
data['Survived'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('Survived')
ax[0].set_ylabel('')
sns.countplot('Survived',data=data,ax=ax[1])
ax[1].set_title('Survived')
plt.show()

많은 승객들이 생존한 사고가 아니라는 증거이다.

training set에서 891명의 승객 중, 오직 350명의 승객만 살아남았다. 즉, 전체 training set 중 **38.4%** 이 사고에서 살아남은 것이다. 데이터에서 더 나은 통찰을 얻기 위해서는 더 조사해야하고, 어떤 카테고리의 승객들의 생존했고 못 했는 지를 알 필요가 있다.

dataset의 다른 feature를 사용하여 생존률을 확인할 것이다. 성별, 승선 항구, 나이 등의 특징들.

먼저, 다른 종류의 feature 들을 이해해야 한다.

## Features의 종류

### Categorical Features:
범주형  변수는 두 개 이상의 범주를 가지고 있고, feature의 각각의 값이 범주로 분류될 수 있다. 예를 들어, 성별은 2개의 범주(남성, 여성)를 가지고 있는 범주형 변수입니다. 그리고 변수들을 분류하거나 순서를 정할 수 없습니다. 이것은 또한 **Nominal Variables(명목 변수)**라고도 합니다.

**Categorical Features in the dataset: Sex,Embarked.**

### Ordinal Features:
순서형 변수는 범주형 값과 비슷하다. 그러나 둘의 차이점은 값들을 상대적으로 순위를 매기거나 분류를 할 수 있다는 것이다. 예를 들어, 만약 **높이** 같은 feature는 **큼, 중간, 작음**의 값을 가진다면, 높이는 순서형 변수이다. 여기에서 변수는 상대적인 분류을 가질 수 있다.

**Ordinal Features in the dataset: PClass**

### Continous Feature:
feature가 feature 열에서 최소,최대값같은 2개의 점에서 값을 가진다면, 연속적이라고 한다.

**Continous Features in the dataset: Age**

## Features 분석하기

## 성별--> Categorical Feature

In [ ]:
data.groupby(['Sex','Survived'])['Survived'].count() #성별별 생존여부

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
data[['Sex','Survived']].groupby(['Sex']).mean().plot.bar(ax=ax[0])
ax[0].set_title('Survived vs Sex')
sns.countplot('Sex',hue='Survived',data=data,ax=ax[1])
ax[1].set_title('Sex:Survived vs Dead')
plt.show()

이것은 흥미롭다. 배에서 남성의 수는 여성의 수보다 훨씬 더 많다. 그럼에도 구해진 여성의 수는 구해진 남성의 수의 거의 2배이다. 배에서의 **여성의 생존률을 약 75%인 반면에, 남성의 경우는 약 18~19%** 이다. 

이것은 모델링에 **매우 중요한** 특징이다. 그러나 이것이 최선일까? 다른 feature들을 확인하자.

## Pclass --> Ordinal Feature

In [ ]:
pd.crosstab(data.Pclass,data.Survived,margins=True).style.background_gradient(cmap='summer_r')

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
data['Pclass'].value_counts().plot.bar(color=['#CD7F32','#FFDF00','#D3D3D3'],ax=ax[0])
ax[0].set_title('Number Of Passengers By Pclass')
ax[0].set_ylabel('Count')
sns.countplot('Pclass',hue='Survived',data=data,ax=ax[1])
ax[1].set_title('Pclass:Survived vs Dead')
plt.show()

사람들은 **돈이 모든 것을 살 수는 없다.** 라고 말한다. 그러나 우리는 분명히 Pclass 1의 승객이 구조에서 매우 높은 우선 순위를 가진다는 것을 알 수 있다. 비록 Pclass 3인 승객의 수가 훨씬 더 높음에도 불구하고, 그들의 생존률은 **25%**로 매우 낮다.

Pclass1 생존률을 약 **63%**이고 Pclass2는 약 **48%**이다. 그래서 돈과 지위가 중요하다. 그런 물질만능적인 세계이다.

다른 흥미로운 관측을 위해 조금 더 깊이 들어가자. **성별과 Pclass** 의 생존율을 함께 확인하자.

In [ ]:
pd.crosstab([data.Sex,data.Survived],data.Pclass,margins=True).style.background_gradient(cmap='summer_r')

In [ ]:
sns.factorplot('Pclass','Survived',hue='Sex',data=data)
plt.show()

우리는 이 경우에 **FactorPlot**을 사용한다, 왜냐하면 범주값 분리를 쉽게 만들 수 있기 때문입니다.

**CrossTab**과 **FactorPlot**을 보면, **Pclass1의 여성**의 생존이 약 **95~96%**가 생존했다는 것을 쉽게 알 수 있고, Pclass1의 94명의 여성 중 오직 3명만 죽었다는 것을 알고 있다.

구조 중에 Pclass와 상관없다는 것과 여성이 최우선이라는 증거입니다. 심지어 Pclass1에서의 남성이 매우 낮은 생존률입니다.
It is evident that irrespective of Pclass, Women were given first priority while rescue. Even Men from Pclass1 have a very low survival rate.

Pclass와 같은 모습은 또한 중요한 feature 입니다. 다른 feature를 분석해봅시다.
Looks like Pclass is also an important feature. Lets analyse other features.

## Age--> Continous Feature


In [ ]:
print('Oldest Passenger was of:',data['Age'].max(),'Years')
print('Youngest Passenger was of:',data['Age'].min(),'Years')
print('Average Age on the ship:',data['Age'].mean(),'Years')

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
sns.violinplot("Pclass","Age", hue="Survived", data=data,split=True,ax=ax[0])
ax[0].set_title('Pclass and Age vs Survived')
ax[0].set_yticks(range(0,110,10))
sns.violinplot("Sex","Age", hue="Survived", data=data,split=True,ax=ax[1])
ax[1].set_title('Sex and Age vs Survived')
ax[1].set_yticks(range(0,110,10))
plt.show()

#### 관측:

1)아이들의 수는 Pclass를 따라 증가하고, 10살 미만의 승객들의 생존율은 Pclass에 상관없이 좋다.

2)Pclass1의 20~50살의 승객의 생존 가능성은 높고, 여성은 훨씬 더 좋다.

3)남성의 경우, 생존 가능성은 나이가 증가할 수록 감소한다.

앞서 본 것처럼, 나이 feature는 177개의 null 값이 있다. 이 NaN 값들을 대체하기 위해, 그것들을 dataset의 나이의 평균으로 할당할 수 있다.

그러나 문제는, 많은 다른 나이를 가진 많은 사람들이 있다는 것이다. 단순히 4살의 아이를 29살의 평균나이로 할당할 수는 없다. 무슨 연령층의 승객이 거짓말하는 지 알아낼 방법이 있을까?

**빙고!!!!**, **이름** feature를 확인할 수 있다. feature를 보면, 군이나 양 같은 표현을 가지고 있다는 것을 알 수 있다. 따라서, 상대적인 그룹에 군과 양의 평균 값을 할당할 수 있다.

**''What's In A Name??''**---> **Feature**  :p

In [ ]:
data['Initial']=0
for i in data:
    data['Initial']=data.Name.str.extract('([A-Za-z]+)\.') #인사표현을 추출한다.

좋다. 여기서는 정규표현식을 사용할 것이다. **[A-Za-z]+)\.** 그래서 이것이 무엇인가. **A-Z 나 a-z** 사이의 문자열 다음에 **.(점)**이 있는 것을 찾는다. 그래서 성공적으로 이름에서 첫글자들을 추출한다.

In [ ]:
pd.crosstab(data.Initial,data.Sex).T.style.background_gradient(cmap='summer_r') #성별과 같은 처음 이름을 확인하기

좋다. Miss 를 나타내는 곳에 Mlle, Mme 와 같은 오타가 있다. 이런 다른 값들을 Miss와 같은 값으로 대체할 것이다.

In [ ]:
data['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'],inplace=True)

In [ ]:
data.groupby('Initial')['Age'].mean() #Initials에 따른 평균 나이를 확인하자

### Filling NaN Ages

In [ ]:
## Cell에서 NaN인 값들을 평균나이로 할당하기
data.loc[(data.Age.isnull())&(data.Initial=='Mr'),'Age']=33
data.loc[(data.Age.isnull())&(data.Initial=='Mrs'),'Age']=36
data.loc[(data.Age.isnull())&(data.Initial=='Master'),'Age']=5
data.loc[(data.Age.isnull())&(data.Initial=='Miss'),'Age']=22
data.loc[(data.Age.isnull())&(data.Initial=='Other'),'Age']=46

In [ ]:
data.Age.isnull().any() #최종적으로 null이 없는 값들

In [ ]:
f,ax=plt.subplots(1,2,figsize=(20,10))
data[data['Survived']==0].Age.plot.hist(ax=ax[0],bins=20,edgecolor='black',color='red')
ax[0].set_title('Survived= 0')
x1=list(range(0,85,5))
ax[0].set_xticks(x1)
data[data['Survived']==1].Age.plot.hist(ax=ax[1],color='green',bins=20,edgecolor='black')
ax[1].set_title('Survived= 1')
x2=list(range(0,85,5))
ax[1].set_xticks(x2)
plt.show()

### Observations:
1)아기(나이<5)들은 많은 수가 생존했다.(여성과 아이 우선 정책).

2)가장 늙은 승객이 생존했다(80세).

3)최대 사망 수는 30-40 나이 그룹에 있었다.

In [ ]:
sns.factorplot('Pclass','Survived',col='Initial',data=data)
plt.show()

여성과 아이 우선 정책은 등급과 상관없이 사실이다.

## Embarked--> Categorical Value

In [ ]:
pd.crosstab([data.Embarked,data.Pclass],[data.Sex,data.Survived],margins=True).style.background_gradient(cmap='summer_r')

### Chances for Survival by Port Of Embarkation

In [ ]:
sns.factorplot('Embarked','Survived',data=data)
fig=plt.gcf()
fig.set_size_inches(5,3)
plt.show()

항구 C에서 생존 가능성이 약 0.55로 가장 높고, 반면에 S가 가장 낮다.

In [ ]:
f,ax=plt.subplots(2,2,figsize=(20,15))
sns.countplot('Embarked',data=data,ax=ax[0,0])
ax[0,0].set_title('No. Of Passengers Boarded')
sns.countplot('Embarked',hue='Sex',data=data,ax=ax[0,1])
ax[0,1].set_title('Male-Female Split for Embarked')
sns.countplot('Embarked',hue='Survived',data=data,ax=ax[1,0])
ax[1,0].set_title('Embarked vs Survived')
sns.countplot('Embarked',hue='Pclass',data=data,ax=ax[1,1])
ax[1,1].set_title('Embarked vs Pclass')
plt.subplots_adjust(wspace=0.2,hspace=0.5)
plt.show()

### Observations:
1)S에서 가장 많은 승객이 탔다. 그들 중 가장 많은 수는 Pclass3이다.

2)C에서 승객들은 생존률이 좋은 것은 운이 좋아보인다. 아마도 이 이유는 Pclass1과 Pclass2 승객들이 모두 구조됐기 때문이다.

3)S 선착장은 대부분 부유한 사람들이 승선한 것 같다. 그럼에도 여기에서 생존율은 낮다. 왜냐하면 Pclass3의 많은 승객들인 약 **81%** 가 생존하지 못 했다.

4)Q 항구는 Pclass3에서 승객들 중 약 95%가 있었다.

In [ ]:
sns.factorplot('Pclass','Survived',hue='Sex',col='Embarked',data=data)
plt.show()

### Observations:

1)Pclass에 상관없이 Pclass1과 Pclass2의 여성들의 생존율은 거의 1이다. (Pclass가 아니라 Embark 같은)

2)S 항구는 Pclass3의 남성과 여성 모두의 생존률이 매우 낮은 것으로 보아, 매우 운이 없어보인다. **(돈이 중요하다.)**

3)Q 항구는 남성들에게 가장 운이 없는 것처럼 보인다. 거의 모든 Pclass3인 사람들처럼.


### Filling Embarked NaN

S 항구에서 승객들이 가장 많이 승선했기 때문에, NaN을 S로 대체한다.

In [ ]:
data['Embarked'].fillna('S',inplace=True)

In [ ]:
data.Embarked.isnull().any()# 최종적으로 NaN 값들은 없다.

## SibSip-->Discrete Feature
이 feature는 사람이 혼자인지 가족들과 있는 지를 나타낸다.

Sibling = brother, sister, stepbrother, stepsister

Spouse = husband, wife 

In [ ]:
pd.crosstab([data.SibSp],data.Survived).style.background_gradient(cmap='summer_r')

In [ ]:
f,ax=plt.subplots(1,2,figsize=(20,8))
sns.barplot('SibSp','Survived',data=data,ax=ax[0])
ax[0].set_title('SibSp vs Survived')
sns.factorplot('SibSp','Survived',data=data,ax=ax[1])
ax[1].set_title('SibSp vs Survived')
plt.close(2)
plt.show()

In [ ]:
pd.crosstab(data.SibSp,data.Pclass).style.background_gradient(cmap='summer_r')

### Observations:

barplot과 factorplot은 승객이 형제없이 혼자 승성했으면, 34.5%의 생존율이라는 것을 보여준다. 그래프는 만약 형제의 수가 증가하면 매우 감소한다. 이것은 이해가 된다. 다시 말해서, 만약 한 가족이 승선한다면, 자기 자신대신에 가족을 구하려고 할 것이다. 놀랍게도 5-8명의 가족의 생존율은 **0%** 다. 이유는 어쩌면 Pclass 일 지도 모른다??

이유는 **Pclass**다. crosstab은 SibSp>3 인 사람이 모두 Pclass3 이라는 것을 보여준다. Pclass3(>3)인 대가족은 모두 죽음에 임박했다는 것이다.

## Parch

In [ ]:
pd.crosstab(data.Parch,data.Pclass).style.background_gradient(cmap='summer_r')

crosstab은 Pclass3에 더 큰 가족들이 있다는 것을 다시 보여준다.

In [ ]:
f,ax=plt.subplots(1,2,figsize=(20,8))
sns.barplot('Parch','Survived',data=data,ax=ax[0])
ax[0].set_title('Parch vs Survived')
sns.factorplot('Parch','Survived',data=data,ax=ax[1])
ax[1].set_title('Parch vs Survived')
plt.close(2)
plt.show()

### Observations:

여기에서 역시 결과들은 꽤 비슷하다. 부모와 함께 탑승한 승객들은 더 큰 생존 기회를 가지고 있다. 그러나 수가 증가하면, 감소한다.

생존 가능성은 배에 1-3명의 부모가 있는 사람은 좋다. 혼자 있는 것은 치명적이다. 그리고 생존 가능성은 배에 >4인 부모들이 있으면 감소한다.

## Fare--> Continous Feature

In [ ]:
print('Highest Fare was:',data['Fare'].max())
print('Lowest Fare was:',data['Fare'].min())
print('Average Fare was:',data['Fare'].mean())

가장 싼 비용은 **0.0** 이다. 무료 호화 탑승이다.

In [ ]:
f,ax=plt.subplots(1,3,figsize=(20,8))
sns.distplot(data[data['Pclass']==1].Fare,ax=ax[0])
ax[0].set_title('Fares in Pclass 1')
sns.distplot(data[data['Pclass']==2].Fare,ax=ax[1])
ax[1].set_title('Fares in Pclass 2')
sns.distplot(data[data['Pclass']==3].Fare,ax=ax[2])
ax[2].set_title('Fares in Pclass 3')
plt.show()

Pclass1에 있는 승객들의 요금이 큰 분포가 있는 것처럼 보이고, 이 분포는 수준이 낮아질 수록 감소한다. 이것은 연속적으로 보인다. 양자화를 사용하여 이산 값으로 바꿀 수 있다.

****## 모든 feature를 위해 껍질 속 관측:
**Sex:** 여성의 생존 가능성이 남성과 비교해서 높다.

**Pclass:** **1등석**이 더 나은 생존율을 갖는다는 뚜렷한 경향이 있다. gives you better chances of survival. **Pclass3**의 생존률이 **매우 낮다.**. **여성**에게는, **Pclass1**의 생존률이 거의 1이고, **Pclass2** 에서의 생존률 또한 매우 높다. **돈이 승리한다!!!**. 

**나이:** 5-18세보다 적은 아이들은 정말로 높은 생존율을 갖는다. 15-35세 사이의 승객들은 많이 죽었다.

**Embarked:** 매우 흥미로운 feature다. **S에서 탄 승객들이 대부분 Pclasss1임에도, C에서의 생존율이 더 낫다.** Q에서의 승객들은 모두 **Pclass3**이었다.

**Parch+SibSp:** 1-2 명의 형제나 1-3명의 부모와 같이 탑승한 것은 혼자나 대가족이 여행하는 것 보다 훨씬 더 높은 생존율을 보여준다.

## Correlation Between The Features

In [ ]:
sns.heatmap(data.corr(),annot=True,cmap='RdYlGn',linewidths=0.2) #data.corr()-->correlation matrix
fig=plt.gcf()
fig.set_size_inches(10,8)
plt.show()

### Heatmap 해석

첫번 째로 기억해야하는 것은 알파벳이나 문자 사이를 연관지을 수 없다는 것이 분명한 것 처럼 오직 numeric feature만 가능하다는 것이다. 그래프를 이해하기 전에, 정확히 무엇이 연관있는 지를 봐야한다.

**POSITIVE CORRELATION:**  만약 **A feature의 상승이 B feature의 상승을 이끌어낸다면, 이 둘은 긍정적으로 관계되어 있다고 한다.** 값이 **1이면 완벽한 positive correlation**이다.

**NEGATIVE CORRELATION:** 만약 **A feature의 상승이 B feature의 감소를 이끌어낸다면, 이 둘은 부정적으로 관계되어 있다고 한다.** 값이 **-1이면 완벽한 negative correlation**이다.

지금 두개의 feature들이 몹시 또는 완벽히 연관되어 있어서, 하나의 상승이 다른 것을 상을을 이끌어 내고 있다. 이것은 feature들이 몹시 비슷한 정보를 포함하고 있는 것과 정보에 변화가 매우 작거나 없다는 것 모두를 의미한다. 이것은 양쪽이 거의 같은 정보를 포함하고 있다는 것으로 **MultiColinearity**으로 알려져 있다.

우리는 **그것들 중 하나**는 불필요한 것처럼 사용해야 한다. 모델을 만들거나 학습하는 동안, 학습 시간을 줄이고 많은 그런 이득을 얻기 위해, 불필요한 feature를 제거하려고 노력해야 한다.

위 heatmap처럼, feature들이 많이 연관되어 있지는 않다는 것을 알 수 있다. 가장 높은 correlation은 **SibSp 와 Parch로 0.41** 이다. 그래서 모든 feature를 사용할 수 있다.

## Part2: Feature Engineering and Data Cleaning

Feature Engineering이 무엇일까?

feature에 대한 하나의 데이터셋이 주어질 때마다, 모든 feature가 중요할 것이라는 것은 필요하지 않다. 아마도 제거돼야하는 많은 불필요한 feature들이 있을 지도 모른다. 또한 다른 feature들로 부터 정보를 관측하고 추출함으로써 새로운 feature들을 얻거나 추가할 수 있다.

한 예는 이름 feature를 사용해서 Initals feature를 얻는 것이다. 어떤 새로운 feature를 얻고 몇몇을 제거해야 하는 지를 확인하자. 또한 예측 모델링을 위한 적합한 형태로 존재하는 적정한 feature로 변환하자.

## Age_band

#### 나이 feature의 문제:
**나이는 연속적인 feature**라고 언급했는데, 머신 러닝 모델에서 연속적인 변수는 문제가 있다.

**예:**만약 **성별**로 운동 선수를 그룹짓거나 정렬한다면, 남성과 여성으로 쉽게 구분할 수 있다.

만약 그들의 **나이**로 그룹화한다면, 어떻게 할 것인가? 만약 30명의 사람들이 있다면, 30살의 값이 있을 지도 모른다. 이것은 문제가 있다.

Binning 이나 Normalisation를 사용해서 **continous values를 categorical values**로 변환할 필요가 있다. binning의 예로 한 나이의 범위를 하나의 단일 구간으로 그룹화하거나 하나의 값으로 할당할 수 있다. 

승객의 최대 나이는 80이었다. 그래서 5개의 구간으로 범위를 나눈다. 80/5=16이므로,
구간의 크기는 16이다.

In [ ]:
data['Age_band']=0
data.loc[data['Age']<=16,'Age_band']=0
data.loc[(data['Age']>16)&(data['Age']<=32),'Age_band']=1
data.loc[(data['Age']>32)&(data['Age']<=48),'Age_band']=2
data.loc[(data['Age']>48)&(data['Age']<=64),'Age_band']=3
data.loc[data['Age']>64,'Age_band']=4
data.head(2)

In [ ]:
data['Age_band'].value_counts().to_frame().style.background_gradient(cmap='summer')#checking the number of passenegers in each band

In [ ]:
sns.factorplot('Age_band','Survived',data=data,col='Pclass')
plt.show()

생존율은 Pclass와 상관없이 나이가 증가함에 따라 감소한다.

## Family_Size and Alone
이 쯤에서, 가족수와 개인여부 라고 부를 수 있는 새로운 feature를 만들고 분석할 수 있다. 이 feature는 parch와 sibsp 의 합니다. 우리가 생존률이 승객들의 가족 수와 연관이 있는 지 확인할 수 있는 조합된 데이터를 얻게 된다. 개인은 승객이 혼자인지 아닌지를 나타낸다.

In [ ]:
data['Family_Size']=0
data['Family_Size']=data['Parch']+data['SibSp']#family size
data['Alone']=0
data.loc[data.Family_Size==0,'Alone']=1#Alone

f,ax=plt.subplots(1,2,figsize=(18,6))
sns.factorplot('Family_Size','Survived',data=data,ax=ax[0])
ax[0].set_title('Family_Size vs Survived')
sns.factorplot('Alone','Survived',data=data,ax=ax[1])
ax[1].set_title('Alone vs Survived')
plt.close(2)
plt.close(3)
plt.show()

**가족수=0은 승객이 혼자인 것을 의미한다.** 분명히, 만약 혼자이거나 가족수=0 이면, 생존률이 매우 낮다. 가족수>4 이면, 생존율은 역시 감소한다. 이것은 또한 모델에 중요한 feature가 되는 것처럼 보인다. 이것을 더 조사하자.

In [ ]:
sns.factorplot('Alone','Survived',data=data,hue='Sex',col='Pclass')
plt.show()

가족과 있는 것보다 혼자인 여성의 생존율이 높은 Pclass3를 제외하면, 혼자있는 것은 성별과 Pclass와 상관없이 해롭다.

## Fare_Range

요금도 연속적인 feature이므로, ordinal value로 변환할 필요가 있다. 이것을 위해 **pands.qcut**을 사용할 것이다.

**qcut**는 우리가 넘겨준 수에 따라서 값을 나누고 정렬한다. 그래서 우리가 5개의 구간을 넘겨준다면, 구간이나 값의 범위를 5개로 나누어서 동등하게 값을 정렬할 것이다.

In [ ]:
data['Fare_Range']=pd.qcut(data['Fare'],4)
data.groupby(['Fare_Range'])['Survived'].mean().to_frame().style.background_gradient(cmap='summer_r')

위에서 살펴본 것처럼, **요금 범위가 증가함에 따라 생존율이 증가한다**라는 것을 분명히 알 수 있다.

요금 범위를 그대로 넘길 수는 없다. 이것을 **age_band**에서 했던 것 처럼 singleton values 로 나눠야 한다.

In [ ]:
data['Fare_cat']=0
data.loc[data['Fare']<=7.91,'Fare_cat']=0
data.loc[(data['Fare']>7.91)&(data['Fare']<=14.454),'Fare_cat']=1
data.loc[(data['Fare']>14.454)&(data['Fare']<=31),'Fare_cat']=2
data.loc[(data['Fare']>31)&(data['Fare']<=513),'Fare_cat']=3

In [ ]:
sns.factorplot('Fare_cat','Survived',data=data,hue='Sex')
plt.show()

분명히, Fare_cat이 증가함에 따라, 생존률이 증가한다. 이 feature는 성별을 따라 모델링것과 마찬가지로 중요한 feature가 될 지도 모른다.

## Converting String Values into Numeric

string을 machine learning model에 넘겨줄 수 없기 때문에, 성별, Embarked 등을 numeric values로 바꿀 필요가 있다.

In [ ]:
data['Sex'].replace(['male','female'],[0,1],inplace=True)
data['Embarked'].replace(['S','C','Q'],[0,1,2],inplace=True)
data['Initial'].replace(['Mr','Mrs','Miss','Master','Other'],[0,1,2,3,4],inplace=True)

### Dropping UnNeeded Features

**Name**--> 우리는 name feature를 어떤 categorical value로 바꿀 수 없기 때문에 필요하지 않다.

**Age**--> 우리는 age_band feature가 있기 때문에, age가 필요하지 않다.

**Ticket**--> 카테고리화 할 수 없는 random string이다.

**Fare**--> Fare_cat feature가 있기 때문에 필요없다.

**Cabin**--> 많은 NaN value 들과 많은 승객들이 다양한 cabin을 가지고 있다. 그래서  이것은 쓸모없다.

**Fare_Range**--> 우리는 fate_cat feature를 가지고 있다.

**PassengerId**--> 카테고리화 될 수 없다.

In [ ]:
data.drop(['Name','Age','Ticket','Fare','Cabin','Fare_Range','PassengerId'],axis=1,inplace=True)
sns.heatmap(data.corr(),annot=True,cmap='RdYlGn',linewidths=0.2,annot_kws={'size':20})
fig=plt.gcf()
fig.set_size_inches(18,15)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()

위에 있는 correlation plot에서, 양의 상관 관계인 feature들을 볼 수 있다. **SibSp 와 Family_Size** 그리고 **Parch와 Family_Size** 또, **Alone과 Family_Size**처럼 음의 상관 관계도 있다.

# Part3: Predictive Modeling

EDA 부분에서 몇몇의 통찰력을 얻었다. 그러나 승객이 생존하고 죽는 지에 대해 정확히 예측하거나 얘기할 수는 없다. 그래서 승객이 생존할 것인지 아닌 지를 몇몇의 훌륭한 Classification Algorithm을 사용해서 예측할 것이다. 다음은 모델을 만들기 위해 사용할 algorithm이다.

1)Logistic Regression

2)Support Vector Machines(Linear and radial)

3)Random Forest

4)K-Nearest Neighbours

5)Naive Bayes

6)Decision Tree

7)Logistic Regression

In [ ]:
#importing all the required ML packages
from sklearn.linear_model import LogisticRegression #logistic regression
from sklearn import svm #support vector Machine
from sklearn.ensemble import RandomForestClassifier #Random Forest
from sklearn.neighbors import KNeighborsClassifier #KNN
from sklearn.naive_bayes import GaussianNB #Naive bayes
from sklearn.tree import DecisionTreeClassifier #Decision Tree
from sklearn.model_selection import train_test_split #training and testing data split
from sklearn import metrics #accuracy measure
from sklearn.metrics import confusion_matrix #for confusion matrix

In [ ]:
train,test=train_test_split(data,test_size=0.3,random_state=0,stratify=data['Survived'])
train_X=train[train.columns[1:]]
train_Y=train[train.columns[:1]]
test_X=test[test.columns[1:]]
test_Y=test[test.columns[:1]]
X=data[data.columns[1:]]
Y=data['Survived']

### Radial Support Vector Machines(rbf-SVM)

In [ ]:
model=svm.SVC(kernel='rbf',C=1,gamma=0.1)
model.fit(train_X,train_Y)
prediction1=model.predict(test_X)
print('Accuracy for rbf SVM is ',metrics.accuracy_score(prediction1,test_Y))

### Linear Support Vector Machine(linear-SVM)

In [ ]:
model=svm.SVC(kernel='linear',C=0.1,gamma=0.1)
model.fit(train_X,train_Y)
prediction2=model.predict(test_X)
print('Accuracy for linear SVM is',metrics.accuracy_score(prediction2,test_Y))

### Logistic Regression

In [ ]:
model = LogisticRegression()
model.fit(train_X,train_Y)
prediction3=model.predict(test_X)
print('The accuracy of the Logistic Regression is',metrics.accuracy_score(prediction3,test_Y))

### Decision Tree

In [ ]:
model=DecisionTreeClassifier()
model.fit(train_X,train_Y)
prediction4=model.predict(test_X)
print('The accuracy of the Decision Tree is',metrics.accuracy_score(prediction4,test_Y))


### K-Nearest Neighbours(KNN)

In [ ]:
model=KNeighborsClassifier() 
model.fit(train_X,train_Y)
prediction5=model.predict(test_X)
print('The accuracy of the KNN is',metrics.accuracy_score(prediction5,test_Y))

KNN 모델의 정확도는 **n_neighbours** 특성의 값이 변함에 따라 변한다. 기본 값은 **5** 다. n_neighbours의 다양한 값에 따라 정확성을 확인하자.

In [ ]:
a_index=list(range(1,11))
a=pd.Series()
x=[0,1,2,3,4,5,6,7,8,9,10]
for i in list(range(1,11)):
    model=KNeighborsClassifier(n_neighbors=i) 
    model.fit(train_X,train_Y)
    prediction=model.predict(test_X)
    a=a.append(pd.Series(metrics.accuracy_score(prediction,test_Y)))
plt.plot(a_index, a)
plt.xticks(x)
fig=plt.gcf()
fig.set_size_inches(12,6)
plt.show()
print('Accuracies for different values of n are:',a.values,'with the max value as ',a.values.max())

### Gaussian Naive Bayes

In [ ]:
model=GaussianNB()
model.fit(train_X,train_Y)
prediction6=model.predict(test_X)
print('The accuracy of the NaiveBayes is',metrics.accuracy_score(prediction6,test_Y))

### Random Forests

In [ ]:
model=RandomForestClassifier(n_estimators=100)
model.fit(train_X,train_Y)
prediction7=model.predict(test_X)
print('The accuracy of the Random Forests is',metrics.accuracy_score(prediction7,test_Y))

모델의 정확성은 classifier의 건장함을 결정하는데 유일한 요소는 아니다. classifier가 training data를 과하게 train되었고 test 데이터로 테스트하면 정확성을 90%이다.
The accuracy of a model is not the only factor that determines the robustness of the classifier. Let's say that a classifier is trained over a training data and tested over the test data and it scores an accuracy of 90%.

classifier에게 매우 좋은 정확성인 것 같다. 그러나 모든 새로운 테스트셋에도 90%일까? 답은 **아니다.** 왜냐하면 classifier가 자신을 훈련하는 데 어떤 예제를 사용할 지 결정할 수 없기 때문이다. training과 testing 데이터가 변함에 따라, 정확도 또한 변할 것이다. 증가하거나 감소할 지도 모른다. 이것은 **model variance**로 알려진다.

이것을 극복하고 일반화된 모델을 얻기 위해 우리는 **Cross Validation**을 사용한다.


# Cross Validation

한 번에 여러번, 데이터는 불균형되어 있다. 예를 들어, class1 예제에는 높은 수가 있지 모르나 다른 class 예제에는 더 낮은 수가 있을 지 모른다. 따라서, 우리의 알고리즘을 각각 그리고 dataset의 모든 예제에 대해 훈련시키고 테스트해야 한다. 그러면 데이터셋에 기록된 모든 정확도들을 평균을 얻을 수 있다.

1)K-Fold Cross Validation은 데이터셋을 k-subsets 으로 나누는 것을 처음으로 작동한다.

2)데이터셋을 (k=5) 부분으로 나눈다고 하자. 테스트를 위한 1개의 파트를 남겨두고 4개의 부분에 대해 알고리즘을 훈련한다.

3)각각의 반복에 테스트하는 부분과 다른 부분들에 대해 알고리즘을 훈련하는 부분을 바꾸면서 프로세스를 계속한다. 알고리즘의 평균 정확성을 얻기 위해 정확성과 오류들은 평균낸다.

이것이 K-Fold Cross Validation이라고 불린다.

4)하나의 알고리즘은 몇몇의 트레이닝 데이터에 대한 하나의 데이터셋에 언더피팅될 지도 모른다. 그리고 때떄로 다른 트레이닝 셋에 대해 데이터가 오버피팅될 지도 모른다. 따라서 cross-validation에서는 일반화된 모델을 얻을 수 있다.

In [ ]:
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction
kfold = KFold(n_splits=10, random_state=22) # k=10, split the data into 10 equal parts
xyz=[]
accuracy=[]
std=[]
classifiers=['Linear Svm','Radial Svm','Logistic Regression','KNN','Decision Tree','Naive Bayes','Random Forest']
models=[svm.SVC(kernel='linear'),svm.SVC(kernel='rbf'),LogisticRegression(),KNeighborsClassifier(n_neighbors=9),DecisionTreeClassifier(),GaussianNB(),RandomForestClassifier(n_estimators=100)]
for i in models:
    model = i
    cv_result = cross_val_score(model,X,Y, cv = kfold,scoring = "accuracy")
    cv_result=cv_result
    xyz.append(cv_result.mean())
    std.append(cv_result.std())
    accuracy.append(cv_result)
new_models_dataframe2=pd.DataFrame({'CV Mean':xyz,'Std':std},index=classifiers)       
new_models_dataframe2

In [ ]:
plt.subplots(figsize=(12,6))
box=pd.DataFrame(accuracy,index=[classifiers])
box.T.boxplot()

In [ ]:
new_models_dataframe2['CV Mean'].plot.barh(width=0.8)
plt.title('Average CV Mean Accuracy')
fig=plt.gcf()
fig.set_size_inches(8,5)
plt.show()

classification 정확성은 때떄로 불균형 떄문에 잘못될 수도 있다. confusion matrix의 도움으로 요약된 결과를 얻을 수 있다. confusion matrix는 모델이 잘 못 되고 있는 지, 또는 클래스가 틀리게 예측했는 지를 보여준다.

## Confusion Matrix

이것은 classifier에 의해 만들어진 정확하고 부정확한 분류의 수를 보여준다.

In [ ]:
f,ax=plt.subplots(3,3,figsize=(12,10))
y_pred = cross_val_predict(svm.SVC(kernel='rbf'),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[0,0],annot=True,fmt='2.0f')
ax[0,0].set_title('Matrix for rbf-SVM')
y_pred = cross_val_predict(svm.SVC(kernel='linear'),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[0,1],annot=True,fmt='2.0f')
ax[0,1].set_title('Matrix for Linear-SVM')
y_pred = cross_val_predict(KNeighborsClassifier(n_neighbors=9),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[0,2],annot=True,fmt='2.0f')
ax[0,2].set_title('Matrix for KNN')
y_pred = cross_val_predict(RandomForestClassifier(n_estimators=100),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[1,0],annot=True,fmt='2.0f')
ax[1,0].set_title('Matrix for Random-Forests')
y_pred = cross_val_predict(LogisticRegression(),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[1,1],annot=True,fmt='2.0f')
ax[1,1].set_title('Matrix for Logistic Regression')
y_pred = cross_val_predict(DecisionTreeClassifier(),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[1,2],annot=True,fmt='2.0f')
ax[1,2].set_title('Matrix for Decision Tree')
y_pred = cross_val_predict(GaussianNB(),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[2,0],annot=True,fmt='2.0f')
ax[2,0].set_title('Matrix for Naive Bayes')
plt.subplots_adjust(hspace=0.2,wspace=0.2)
plt.show()

### Interpreting Confusion Matrix

외쪽 대각선은 각각의 클래스에서 만들어진 올바른 예의 수를 보여준다. 반면에 오른쪽 대각선은 잘못 만들어진 예측의 수를 보여준다. rbf-SVN을 위한 첫번째 plot을 보자.

1)올바른 예측의 수는 **491(사망) + 247(생존)**이고, 평균 CV 정확도는 이전에 얻었던 **(491+247)/891 = 82.5%** 이다.

2)**오류**--> 58명의 사망자를 생존자로, 95명의 생존자를 사망자로 잘못 분류했다. 따라서 사망자를 생존자로 예측하는 데 더 많은 실수가 있었다.

모든 행렬을 보면서, rbf-SVM이 사망자를 올바르게 예측할 확률이 더 높으나, NaiveBayes는 생존자를 올바르게 예측할 확률이 더 높다고 할 수 있다.

### Hyper-Parameters Tuning

머신 러닝 모델은 블랙박스같다. 이 블랙박스를 위해서는 몇몇의 기본 파라미터 값들이 있다. 그리고 이 값들은 더 나은 모델을 얻기 위해 조정하고 바꿀 수 있다. SVM 모델의 C와 gamma처럼, 다른 classifiers들의 단순히 다른 파라미터들을hyper-parameter라고 한다. hyper-parameter는 알고리즘의 learning rate를 바꾸기거나 더 나은 모델을 얻기 위해 조절할 수 있다. 이것을 hyper-parameter 튜닝이라고 알려져있다.

2개의 가장 나은 classifier인 SVM과 RandomForest에서 hyper-parameter를 조절할 것이다.

#### SVM

In [ ]:
from sklearn.model_selection import GridSearchCV
C=[0.05,0.1,0.2,0.3,0.25,0.4,0.5,0.6,0.7,0.8,0.9,1]
gamma=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
kernel=['rbf','linear']
hyper={'kernel':kernel,'C':C,'gamma':gamma}
gd=GridSearchCV(estimator=svm.SVC(),param_grid=hyper,verbose=True)
gd.fit(X,Y)
print(gd.best_score_)
print(gd.best_estimator_)

#### Random Forests

In [ ]:
n_estimators=range(100,1000,100)
hyper={'n_estimators':n_estimators}
gd=GridSearchCV(estimator=RandomForestClassifier(random_state=0),param_grid=hyper,verbose=True)
gd.fit(X,Y)
print(gd.best_score_)
print(gd.best_estimator_)

Rbf-SVM의 최고 점수는 **C=0.05, gamma=0.1일 때, 82.82%**이다.
RandomForest에서는, 점수가 약 **n_estimators=900일 때, 81.8%**이다.

> # Ensembling

Ensembling은 모델의 정확도와 성능을 높이는 좋은 방법이다. 간단히 말해서, 하나의 강력한 모델을 만들기 위해 다양하고 단순한 모델들의 조합이다.

우리가 전화를 사고 싶고 다양한 파라미터들을 기반으로 그것에 대해 많은 사람들에게 묻는다고 하자. 그래서 모든 파라미터들을 분석한 후에 하나의 상품에 대한 확실한 판단을 만들 수 있다. 이것이 **Ensembling**이다. Ensembling은 모델의 안정성을 증가시킨다. Ensembling은 다음과 같은 방법으로 행해진다.

1)Voting Classifier

2)Bagging

3)Boosting

## Voting Classifier

많은 다른 단순한 머신 러닝 모델로 부터 예측들을 조합하는 가장 단순한 방법이다. 이것은 모든 서브모델의 예측에 기반하여 예측 평균 결과를 가져온다. 서브 모델이나 기본 모델들은 모두 다른 형식이 있다.

In [ ]:
from sklearn.ensemble import VotingClassifier
ensemble_lin_rbf=VotingClassifier(estimators=[('KNN',KNeighborsClassifier(n_neighbors=10)),
                                              ('RBF',svm.SVC(probability=True,kernel='rbf',C=0.5,gamma=0.1)),
                                              ('RFor',RandomForestClassifier(n_estimators=500,random_state=0)),
                                              ('LR',LogisticRegression(C=0.05)),
                                              ('DT',DecisionTreeClassifier(random_state=0)),
                                              ('NB',GaussianNB()),
                                              ('svm',svm.SVC(kernel='linear',probability=True))
                                             ], 
                       voting='soft').fit(train_X,train_Y)
print('The accuracy for ensembled model is:',ensemble_lin_rbf.score(test_X,test_Y))
cross=cross_val_score(ensemble_lin_rbf,X,Y, cv = 10,scoring = "accuracy")
print('The cross validated score is',cross.mean())

## Bagging

Bagging은 일반적인 ensemble 방법이다. 데이터셋에 대한 작은 부분들에 대해 비슷한 classifier를 적용하고, 모든 예측의 평균을 가지고 작동한다. 평균 때문에, variance가 감소한다. Voting Classifier와 다르게, Bagging은 비슷한 Classifier를 이용한다.

#### Bagged KNN

Bagging은 높은 variance를 가진 모델들에 가장 좋다. 이것의 예는 결정트리나 랜덤포레스트가 될 수 있다. 우리는 **n_neghbours**의 작은 값으로 KNN을 사용할 수 있다.

In [ ]:
from sklearn.ensemble import BaggingClassifier
model=BaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=3),random_state=0,n_estimators=700)
model.fit(train_X,train_Y)
prediction=model.predict(test_X)
print('The accuracy for bagged KNN is:',metrics.accuracy_score(prediction,test_Y))
result=cross_val_score(model,X,Y,cv=10,scoring='accuracy')
print('The cross validated score for bagged KNN is:',result.mean())

#### Bagged DecisionTree


In [ ]:
model=BaggingClassifier(base_estimator=DecisionTreeClassifier(),random_state=0,n_estimators=100)
model.fit(train_X,train_Y)
prediction=model.predict(test_X)
print('The accuracy for bagged Decision Tree is:',metrics.accuracy_score(prediction,test_Y))
result=cross_val_score(model,X,Y,cv=10,scoring='accuracy')
print('The cross validated score for bagged Decision Tree is:',result.mean())

## Boosting

Boosting은 classifier의 연속적인 학습을 사용하는 ensemblig 기술이다. 이것은 약한 model.Boosting의 이 다음처럼 작용하는 단게적 증가이다.

모델은 완벽한 데이터셋에 처음으로 훈련된다. 모델은 몇몇은 잘못된 반면에 몇몇은 올바른 예를 가질 것이다. 다음 반복에서, 학습은 잘못 예측된 예제에 집중하거나 그것에 더 많은 가중치를 주는 것에 집중할 것이다. 따라서 틀린 예제를 올바르게 예측하려고 노력할 것이다. 정확성의 한계까지 도달할 때까지, 이런 반복적인 처리를 계속하고, 새로운 classifier는 모델에 추가된다.

#### AdaBoost(Adaptive Boosting)

이런 경우에, 약한 learner나 estimator를 가지고 있는 것은 결정 트리다. 그러나 우리가 선택한 알고리즘에 기본 base_estimator를 바꿀 수 있다.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada=AdaBoostClassifier(n_estimators=200,random_state=0,learning_rate=0.1)
result=cross_val_score(ada,X,Y,cv=10,scoring='accuracy')
print('The cross validated score for AdaBoost is:',result.mean())

#### Stochastic Gradient Boosting

여기도 또한 약한 learner가 결정트리이다.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
grad=GradientBoostingClassifier(n_estimators=500,random_state=0,learning_rate=0.1)
result=cross_val_score(grad,X,Y,cv=10,scoring='accuracy')
print('The cross validated score for Gradient Boosting is:',result.mean())

#### XGBoost

In [ ]:
import xgboost as xg
xgboost=xg.XGBClassifier(n_estimators=900,learning_rate=0.1)
result=cross_val_score(xgboost,X,Y,cv=10,scoring='accuracy')
print('The cross validated score for XGBoost is:',result.mean())

우리는 가장 AdaBoost에 가장 높은 정확도를 얻었다. 우리는 Hyper-parameter 튜닝을 통해 정확도를 증가시킬 것이다.

#### Hyper-Parameter Tuning for AdaBoost

In [ ]:
n_estimators=list(range(100,1100,100))
learn_rate=[0.05,0.1,0.2,0.3,0.25,0.4,0.5,0.6,0.7,0.8,0.9,1]
hyper={'n_estimators':n_estimators,'learning_rate':learn_rate}
gd=GridSearchCV(estimator=AdaBoostClassifier(),param_grid=hyper,verbose=True)
gd.fit(X,Y)
print(gd.best_score_)
print(gd.best_estimator_)

AdaBoost를 통해 얻을 수 있는 최고 정확도는 **n_estimators=200, learning_rate=0.05일 때, 83.16%**이다.

### Confusion Matrix for the Best Model

In [ ]:
ada=AdaBoostClassifier(n_estimators=200,random_state=0,learning_rate=0.05)
result=cross_val_predict(ada,X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,result),cmap='winter',annot=True,fmt='2.0f')
plt.show()

## Feature Importance

In [ ]:
f,ax=plt.subplots(2,2,figsize=(15,12))
model=RandomForestClassifier(n_estimators=500,random_state=0)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8,ax=ax[0,0])
ax[0,0].set_title('Feature Importance in Random Forests')
model=AdaBoostClassifier(n_estimators=200,learning_rate=0.05,random_state=0)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8,ax=ax[0,1],color='#ddff11')
ax[0,1].set_title('Feature Importance in AdaBoost')
model=GradientBoostingClassifier(n_estimators=500,learning_rate=0.1,random_state=0)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8,ax=ax[1,0],cmap='RdYlGn_r')
ax[1,0].set_title('Feature Importance in Gradient Boosting')
model=xg.XGBClassifier(n_estimators=900,learning_rate=0.1)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8,ax=ax[1,1],color='#FD0F00')
ax[1,1].set_title('Feature Importance in XgBoost')
plt.show()

우리는 다양한 랜덤포레스트나 adaBoost 등 같은 다양한 classifiers들을 위한 중요한 feature를 알 수 있다.

#### Observations:

1)공통의 중요한 feature들은 Initial, Fare_cat, Pclass, Family_Size이다.

2)성별 feature는 중요성을 주는 것 같지 않다. 우리가 이전에 알았던 것처럼 충격적이다. Pclass와 조합된 성별이 매우 좋은 차이를 만들어내는 요소를 주고 있었다. 성별은 오직 랜덤포레스트에서만 중요한 것처럼 보인다.

그러나, feature Initial을 보면, 많은 classifier에서 최고이다. 우리는 성별과 Initial사이에 이미 양의 상관관계를 봤었다. 그래서 둘 다 gender를 의미하는 것이다.

3)비슷하게, Pclass와 Fare_cat은 승객의 지위를 나타내고, Family_Size는 개인,가족,형제를 나타낸다.

I hope all of you did gain some insights to Machine Learning. Some other great notebooks for Machine Learning are:
1) For R:[Divide and Conquer by Oscar Takeshita](https://www.kaggle.com/pliptor/divide-and-conquer-0-82297/notebook)

2)For Python:[Pytanic by Heads and Tails](https://www.kaggle.com/headsortails/pytanic)

3)For Python:[Introduction to Ensembling/Stacking by Anisotropic](https://www.kaggle.com/arthurtok/introduction-to-ensembling-stacking-in-python)

### Thanks a lot for having a look at this notebook. If you found this notebook useful, **Do Upvote**.
